# 빅콘테스트 퓨처스리그 : KBO 정규시즌 팀별 승률, 타율, 방어율 예측

2020.09.24 목요일 김채형

## Crawling - KBO 기록 (선수 별)

In [9]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re

### 타자

In [3]:
batter = pd.read_csv('../../data/batter_left.csv')
batter

year  month   P_ID T_ID  PA  AB  HIT  H2  H3  HR  ...  VS_T_ID_LG  \
0    2020      7  65207   LG   2   2    2   0   0   0  ...           0   
1    2020      7  69992   NC   2   2    2   2   0   0  ...           2   
2    2020      7  50066   KT   1   1    1   0   0   1  ...           3   
3    2020      7  61554   HH   1   1    1   0   0   0  ...           4   
4    2020      7  78603   HT  19  17   12   3   0   0  ...           0   
..    ...    ...    ...  ...  ..  ..  ...  ..  ..  ..  ...         ...   
685  2020      9  63435   SS   4   4    0   0   0   0  ...           2   
686  2020      9  69418   SS   0   0    0   0   0   0  ...           2   
687  2020      9  61457   SS   0   0    0   0   0   0  ...           2   
688  2020      9  66707   HH   7   7    0   0   0   0  ...           2   
689  2020      9  65522   LT   0   0    0   0   0   0  ...           6   

     VS_T_ID_LT  VS_T_ID_NC  VS_T_ID_OB  VS_T_ID_SK  VS_T_ID_SS  VS_T_ID_WO  \
0             3           2           6           3           3           0   
1             3           0           1           3           1           3   
2             0           6           0           1           2           3   
3             3           0           3           6           2           0   
4             1           3           3           0           5           3   
..          ...         ...         ...         ...         ...         ...   
685           2           3           4           2           0           2   
686           2           3           4           2           0           2   
687           2           3           4           2           0           2   
688           2           2           5           2           3           4   
689           0           5           0           2           2           2   

     H_A_AWAY  H_A_HOME  BAT_ORDER  
0          16         8        3.0  
1          12        10        6.0  
2           9        13        3.0  
3          14        11        2.0  
4           9        14        1.0  
..        ...       ...        ...  
685        11        14        NaN  
686        11        14        7.0  
687        11        14        NaN  
688        10        16        8.0  
689        15        12        NaN  

[690 rows x 29 columns]

In [4]:
batterPID = list(set(batter['P_ID']))

In [5]:
batterdf = pd.DataFrame(columns=('P_ID', 'year', 'month', 'AVG1', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP', 'AVG2'))
batterdf

Empty DataFrame
Columns: [P_ID, year, month, AVG1, PA, AB, R, H, 2B, 3B, HR, RBI, SB, CS, BB, HBP, SO, GDP, AVG2]
Index: []

In [6]:
# 크롬드라이버 시작
path = '/Users/kimchaehyeong/Downloads/chromedriver'
driver = webdriver.Chrome(path)

# 모든 선수에 대하여 반복
for pid in batterPID:
    
    url = 'https://www.koreabaseball.com/Record/Player/HitterDetail/Daily.aspx?playerId=' + str(pid)
    driver.get(url)
    
    # 웹페이지 크롤링 및 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 월
    mtotal = soup.select('#contents > div.sub-content > div.player_records > div.tbl-type02 > table > thead > tr')
    month = []
    for m in mtotal:
        p = re.compile(r'\d+(?=월)')
        mm = p.findall(str(m))
        month += mm
    
    # 합계
    total = soup.select('#contents > div.sub-content > div.player_records > div.tbl-type02 > table > tfoot > tr')
    try:
        # 월 별로 기록 데이터프레임에 추가
        for i in range(len(total)):
            p = re.compile(r'-|\d+.?\d*(?=</th>)')
            result = p.findall(str(total[i]))
            batterdf = batterdf.append(
                {'P_ID': int(pid), 'year': 2020, 'month': month[i], 
                 'AVG1' : result[0], 'PA' : result[1], 'AB' : result[2], 'R': result[3], 'H' : result[4], 
                 '2B' : result[5], '3B' : result[6], 'HR' : result[7], 'RBI' : result[8], 'SB' : result[9], 
                 'CS' : result[10], 'BB' : result[11], 'HBP' : result[12], 'SO' : result[13], 'GDP' : result[14], 
                 'AVG2' : result[15]}, 
                ignore_index=True)
    
    # 에러 발생한 선수코드 확인
    except:
        print(pid)
        pass
    
    time.sleep(3)
    
# 크롬드라이버 종료
driver.quit()

In [7]:
batterdf

P_ID  year month   AVG1   PA   AB   R   H  2B 3B HR RBI SB CS  BB HBP  \
0     76290  2020     5  0.391  101   92  20  36  12  1  1  16  0  0   9   0   
1     76290  2020     6  0.290  114  107  15  31   5  0  4  16  0  1   5   0   
2     76290  2020     7  0.352  106   91  20  32   5  0  9  29  0  1  14   0   
3     76290  2020     8  0.362  106   94  15  34   5  0  6  21  0  0  10   2   
4     76290  2020     9  0.403   74   62  13  25   4  0  1  24  0  0  10   0   
...     ...   ...   ...    ...  ...  ...  ..  ..  .. .. ..  .. .. ..  ..  ..   
1068  64504  2020     9  0.286   19   14   2   4   1  0  0   2  1  0   5   0   
1069  73209  2020     5  0.200   43   40   5   8   1  0  1   4  0  0   1   0   
1070  73209  2020     6  0.333   36   33   3  11   1  0  3   4  0  0   2   0   
1071  73209  2020     7  0.268   87   82   9  22   1  0  2  12  0  0   2   1   
1072  73209  2020     8  0.327   56   49   4  16   2  0  0   6  0  0   7   0   

      SO GDP   AVG2  
0      7   2  0.391  
1     10   3  0.337  
2     10   3  0.341  
3      8   0  0.346  
4     11   0  0.354  
...   ..  ..    ...  
1068   4   0  0.288  
1069   6   1  0.200  
1070   5   1  0.260  
1071  21   3  0.265  
1072  12   1  0.279  

[1073 rows x 19 columns]

In [8]:
batterdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1073 entries, 0 to 1072
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   P_ID    1073 non-null   object
 1   year    1073 non-null   object
 2   month   1073 non-null   object
 3   AVG1    1073 non-null   object
 4   PA      1073 non-null   object
 5   AB      1073 non-null   object
 6   R       1073 non-null   object
 7   H       1073 non-null   object
 8   2B      1073 non-null   object
 9   3B      1073 non-null   object
 10  HR      1073 non-null   object
 11  RBI     1073 non-null   object
 12  SB      1073 non-null   object
 13  CS      1073 non-null   object
 14  BB      1073 non-null   object
 15  HBP     1073 non-null   object
 16  SO      1073 non-null   object
 17  GDP     1073 non-null   object
 18  AVG2    1073 non-null   object
dtypes: object(19)
memory usage: 159.4+ KB


In [9]:
batterdf.to_csv('../../data/kbo_record_batter_byPID.csv')

### 투수

In [10]:
pitcher = pd.read_csv('../../data/pitcher_left.csv', encoding='EUC-KR')
pitcher

year  month   P_ID T_ID   PA   AB  HIT  H2  H3  HR  BB  HP  KK  BK    AVG
0    2020      7  65057   KT    0    0    0   0   0   0   0   0   0   0  0.000
1    2020      7  63961   HH    2    0    0   0   0   0   2   0   1   0  0.000
2    2020      7  72447   OB    1    1    1   0   1   0   0   0   0   0  1.000
3    2020      7  64805   OB    2    2    1   0   0   0   0   0   0   0  0.500
4    2020      7  61145   LG    2    1    1   0   0   0   1   0   0   0  1.000
..    ...    ...    ...  ...  ...  ...  ...  ..  ..  ..  ..  ..  ..  ..    ...
568  2020      9  50404   SS  113  103   26   3   0   2   9   1  25   0  0.252
569  2020      9  69446   SS   82   76   26   4   0   3   5   1  10   0  0.342
570  2020      9  68419   SS  102   95   26   6   0   0   7   0  28   0  0.274
571  2020      9  68135   LG   83   81   29   5   1   0   0   2   9   0  0.358
572  2020      9  77318   HH  113   99   29  10   0   1  13   1  15   0  0.293

[573 rows x 15 columns]

In [11]:
pitcher.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    573 non-null    int64  
 1   month   573 non-null    int64  
 2   P_ID    573 non-null    int64  
 3   T_ID    573 non-null    object 
 4   PA      573 non-null    int64  
 5   AB      573 non-null    int64  
 6   HIT     573 non-null    int64  
 7   H2      573 non-null    int64  
 8   H3      573 non-null    int64  
 9   HR      573 non-null    int64  
 10  BB      573 non-null    int64  
 11  HP      573 non-null    int64  
 12  KK      573 non-null    int64  
 13  BK      573 non-null    int64  
 14  AVG     573 non-null    float64
dtypes: float64(1), int64(13), object(1)
memory usage: 67.3+ KB


In [14]:
pitcherPID = list(set(pitcher['P_ID']))

In [15]:
pitcherdf = pd.DataFrame(columns=('P_ID', 'year', 'month', 'ERA1', 'TBF', 'IP', 'H', 'HR', 'BB', 'HBP', 'SO', 'R', 'ER', 'ERA2'))
pitcherdf

Empty DataFrame
Columns: [P_ID, year, month, ERA1, TBF, IP, H, HR, BB, HBP, SO, R, ER, ERA2]
Index: []

In [16]:
# 크롬드라이버 시작
path = '/Users/kimchaehyeong/Downloads/chromedriver'
driver = webdriver.Chrome(path)
    
# 모든 선수에 대하여 반복
for pid in pitcherPID:
    
    url = 'https://www.koreabaseball.com/Record/Player/PitcherDetail/Daily.aspx?playerId=' + str(int(pid))
    driver.get(url)
    
    # 웹페이지 크롤링 및 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
        
    # 월
    mtotal = soup.select('#contents > div.sub-content > div.player_records > div.tbl-type02 > table > thead > tr')
    month = []
    for m in mtotal:
        p = re.compile(r'\d+(?=월)')
        mm = p.findall(str(m))
        month += mm
    
    # 합계
    total = soup.select('#contents > div.sub-content > div.player_records > div.tbl-type02 > table > tfoot > tr')
    try:
        # 월 별로 기록 데이터프레임에 추가
        for i in range(len(total)):
            p = re.compile(r'-|\d+.?\d*(?=</th>)')
            result = p.findall(str(total[i]))
            pitcherdf = pitcherdf.append(
                {'P_ID': int(pid), 'year': 2020, 'month': month[i], 
                 'ERA1' : result[0],'TBF' : result[1], 'IP' : result[2], 'H' : result[3], 'HR' : result[4], 
                 'BB' : result[5], 'HBP' : result[6], 'SO' : result[7], 'R' : result[8], 'ER' : result[9], 
                 'ERA2' : result[10]}, 
                ignore_index=True)
    
    # 에러 발생 선수코드 확인
    except:
        print(pid)
        pass
    
    time.sleep(3)
    
# 크롬드라이버 종료
driver.quit()

In [17]:
pitcherdf

P_ID  year month   ERA1  TBF   IP   H HR  BB HBP  SO   R  ER    ERA2
0    63492  2020     7  67.50    7  2/3   4  2   1   0   1   5   5   67.50
1    63492  2020     8      -    6    0   4  0   1   0   0   6   5  135.00
2    77318  2020     5   6.93  128  2/3  34  2  18   4  31  21  19    6.93
3    77318  2020     6   4.85   62   13  19  3   7   0  15   7   7    6.21
4    77318  2020     7   2.78  143  2/3  22  3  15   1  36  11  11    4.54
..     ...   ...   ...    ...  ...  ...  .. ..  ..  ..  ..  ..  ..     ...
902  66557  2020     9  27.00    2  1/3   1  0   0   0   0   1   1    5.14
903  79358  2020     5   3.68   36  1/3  10  0   5   0   5   3   3    3.68
904  79358  2020     6   6.00   16    3   5  1   2   0   2   2   2    4.35
905  79358  2020     7   6.14   35  1/3  11  1   2   0   6   5   5    5.09
906  79358  2020     8  36.00    6    1   4  1   0   0   1   4   4    6.75

[907 rows x 14 columns]

In [18]:
pitcherdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 0 to 906
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   P_ID    907 non-null    object
 1   year    907 non-null    object
 2   month   907 non-null    object
 3   ERA1    907 non-null    object
 4   TBF     907 non-null    object
 5   IP      907 non-null    object
 6   H       907 non-null    object
 7   HR      907 non-null    object
 8   BB      907 non-null    object
 9   HBP     907 non-null    object
 10  SO      907 non-null    object
 11  R       907 non-null    object
 12  ER      907 non-null    object
 13  ERA2    907 non-null    object
dtypes: object(14)
memory usage: 99.3+ KB


In [19]:
pitcherdf.to_csv('../../data/kbo_record_pitcher_byPID.csv')